In [24]:
import numpy as np
import MDAnalysis as mda
from tqdm import tqdm
import subprocess
from collections import Counter
import os
main_dir = os.getcwd()

def parse_file(filename):
    atoms_info = []
    in_atoms_section = False
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if line == '[ atoms ]':
                in_atoms_section = True
                continue
            if in_atoms_section:
                if line.startswith('['):
                    break
                if not line.startswith(';') and line:
                    parts = line.split()
                    atom_info = {
                        'atom_name': parts[4],
                        'resname': parts[3],
                        'resid': int(parts[2])
                    }
                    atoms_info.append(atom_info)
    return atoms_info

def create_random_structure_from_file(filename, outdir):
    atoms_info = parse_file(filename)
    sol = mda.Universe.empty(len(atoms_info),
                         n_residues=1,
                         atom_resindex=[0]*len(atoms_info),
                         residue_segindex=[0],
                         trajectory=True)
    sol.add_TopologyAttr('name', [info['atom_name'] for info in atoms_info]*1)
    sol.add_TopologyAttr('resname', [atoms_info[0]['resname']]*1)
    sol.add_TopologyAttr('resid', [atoms_info[0]['resid']]*1)
    poslis=[]
    for i in range(len(sol.atoms)):
        poslis.append([np.random.rand()*10, np.random.rand()*10, np.random.rand()*10])
    
    positions = np.array(poslis)
    sol.atoms.positions = positions
    sol.atoms.write(outdir)


m3_ffs               = '/local/lborge01/m3_ffs/'
partition_tools_base = "/local/lborge01/Martini_Partition_Source/"
mdps                 = "/local/lborge01/Martini_Partition_Source/mdps_to_fix_initial_structure/"

In [43]:
fragments = [
'ALC',
'Amine2_link_alcohol',
'Amine_branched',
'Amine_carbamat',
'Amine_ester_disulfied',
'Amine_ethers',
'Amine_link_alcohol',
'Amine_link_ester',
'Amine_link_ether',
'DAP_amine',
'DMG',
'Double_ester_long',
# 'Double_ester_long_branched(missing)',
'Ester_branched3',
'Ester_branched_long_sat',
'Ester_branched_long_unsat',
# 'Ester_tail1(missing)',
# 'Ester_tail2(missing)',
# 'Ether_link_sat(missing)',
# 'Ether_link_unsat(missing)',
'KC1_amine',
'KC2_amine',
'MC3_amine',
'Piperazine',
'SC1_ester_SC1',
]

fragments = [
# 'ALC',
# 'Amine2_link_alcohol',
# 'Amine_branched',
# 'Amine_carbamat',
# 'Amine_ester_disulfied',
# 'Amine_ethers',
# 'Amine_link_alcohol',
# 'Amine_link_ester',
# 'Amine_link_ether',
# 'DAP_amine',
# 'DMG',
# 'Double_ester_long',
# # 'Double_ester_long_branched(missing)',
# 'Ester_branched3',
# 'Ester_branched_long_sat',
# 'Ester_branched_long_unsat',
# # 'Ester_tail1(missing)',
# # 'Ester_tail2(missing)',
# # 'Ether_link_sat(missing)',
# # 'Ether_link_unsat(missing)',
# 'KC1_amine',
# 'KC2_amine',
# 'MC3_amine',
# 'Piperazine',
'SC1_ester_SC1',
]

In [38]:
def GET_STARTING_STRUCTURE_REEEEEE(filename):
    structure='initial.gro'

    create_random_structure_from_file(filename, structure)

    
    ## Write a topology file.
    header=[f'#include "{m3_ffs}/martini_v3.0.0.itp"\n',
            f'#include "{m3_ffs}/martini_v3.0.0_solvents_v1.itp"\n',
            f'#include "{m3_ffs}/martini_v3.0.0_ions_v1.itp"\n',
            f'#include "{filename}"\n',
            '[system]\n',
            f'FIX THIS SHIT STRUCTURE REEEEEEEE: {filename}\n',
            '\n',
            '[ molecules ]\n',
            ]
    counts = Counter(mda.Universe(structure).atoms.residues.resnames)
    for item, count in counts.items():
        header.append(f"{item}    {count}\n")
    with open(f'topol.top', 'w+') as topout:
        for line in header:
            topout.write(line)    

    
    with open('./log.out', 'a') as output_file:
        subprocess.call(f'gmx editconf -f {structure} -o box.gro -box 11 11 11 -bt dodecahedron'
                , shell=True, stdout=output_file, stderr=subprocess.STDOUT)
        subprocess.call(f'gmx solvate -cp box.gro -cs {partition_tools_base}/solvents/water.gro -o watered.gro -p topol.top'
                , shell=True, stdout=output_file, stderr=subprocess.STDOUT)
        subprocess.call(f"gmx grompp -f {mdps}/min.mdp -c watered.gro -p topol.top -o min.tpr -maxwarn 5", 
                        shell = True, stdout=output_file, stderr=subprocess.STDOUT)
        subprocess.call(f"gmx mdrun -deffnm min -v -nt 5 -pin on -pinoffset 0", 
                        shell = True, stdout=output_file, stderr=subprocess.STDOUT)
        subprocess.call(f"gmx grompp -f {mdps}/rel.mdp -c min.gro -p topol.top -o rel.tpr -maxwarn 5", 
                            shell = True, stdout=output_file, stderr=subprocess.STDOUT)
        subprocess.call(f"gmx mdrun -deffnm rel -v -nt 5 -pin on -pinoffset 0", 
                            shell = True, stdout=output_file, stderr=subprocess.STDOUT)

    mda.Universe('rel.gro').select_atoms('not resname W').atoms.write('structure.gro')
    subprocess.call(f"rm initial.gro rel.* min.* box.* topol.top mdout.mdp watered.gro \#*", 
                        shell = True)

In [47]:
os.chdir(main_dir)
for fragment in tqdm(fragments):
    workdir = f"/local/lborge01/LNP/PartitionTesting/Fragments/{fragment}/"
    os.chdir(workdir)
    print(f"Working on: {fragment}")
    try:
        GET_STARTING_STRUCTURE_REEEEEE('itp.itp')
    except:
        print(f'Something wrong happened with fragment: {fragment}. Passing.')
        pass
    os.chdir(main_dir)


  0%|                                             | 0/1 [00:00<?, ?it/s]

Working on: SC1_ester_SC1


100%|█████████████████████████████████████| 1/1 [00:11<00:00, 11.61s/it]
